# Lauren Bassett 
# jme9rt 
# DS5001 Final Project  

*The goal of your final project is to apply and integrate what you have learned in this course to
create ​a digital critical edition of a corpus​ that will support exploration of the cultural contents of
that corpus. Cultural content is broadly conceived​—it may be about language use, social events,
cultural categories, sentiments, identity, taste, etc.*

## Part 0: Import Statements/Reading in Files

In [68]:
import pandas as pd 
import numpy as np
%matplotlib inline
import re
from glob import glob
import re
import nltk
from scipy.linalg import eigh

nltk.download('punkt')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
xmas_carol_text = 'christmas-carol.txt'
wise_man_text = 'wise-man.txt'
OHCO = ['title', 'chap_num', 'para_num', 'sent_num', 'token_num']

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]   Package averaged_perceptron_tagger is already up-to-
[nltk_data]       date!
[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\laure\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


## Part 1: Convert

In [12]:
#-------
# Christmas Carol
xmas = open(xmas_carol_text, 'r', encoding='utf-8-sig').readlines()
df = pd.DataFrame(xmas, columns=['line_str'])
df.index.name = 'line_num'
df.line_str = df.line_str.str.strip()
title = df.loc[0].line_str.replace('The Project Gutenberg EBook of ', '')
df['title'] = title

#--------
# The Other Wise man
wise = open(wise_man_text, 'r', encoding='utf-8-sig').readlines()
df2 = pd.DataFrame(wise, columns=['line_str'])
df2.index.name = 'line_num'
df2.line_str = df2.line_str.str.strip()
title = df2.loc[0].line_str.replace('The Project Gutenberg eBook,', '')
df2['title'] = title

There are illustrations in the text, and I want to make sure to remove the captions for all the illustrations. However, there are a handful that take up multiple lines. To remove these from the corpus, I found it easier to remove these first, then continue with the rest of the data cleansing. 

In [13]:
#------
# A Christmas Carol
start = df.line_str.str.match(r'\[.*[^\]]$')
df.loc[start]


,line_str,title
line_num,,
40,"[Illustration: _""How now?"" said Scrooge, caust...","A Christmas Carol, by Charles Dickens"
570,[Illustration: _Bob Cratchit went down a slide...,"A Christmas Carol, by Charles Dickens"
669,[Illustration: _Nobody under the bed; nobody i...,"A Christmas Carol, by Charles Dickens"
971,[Illustration: _The air was filled with phanto...,"A Christmas Carol, by Charles Dickens"
1364,[Illustration: HE PRODUCED A DECANTER OF CURIO...,"A Christmas Carol, by Charles Dickens"
1463,[Illustration: _Then old Fezziwig stood out to...,"A Christmas Carol, by Charles Dickens"
2458,[Illustration: _The way he went after that plu...,"A Christmas Carol, by Charles Dickens"
2735,[Illustration:,"A Christmas Carol, by Charles Dickens"
3527,"[Illustration: _""It's I, your uncle Scrooge. I...","A Christmas Carol, by Charles Dickens"


In [14]:
big_quotes = []
for index, row in df.loc[start].iterrows():
    big_quotes.append(index)
    end = 1
    while end != -1: 
        #Continue to iterate through each line of the dataframe until the illustration is over, which is marked with the ']' character. 
        # Keep track of each line included, all of these need to be removed. 
        big_quotes.append(index+end)
        if re.search('\]', df.iloc[index+end]['line_str']):
            end = -1
        else:
            end = end+1
df.drop(big_quotes, axis=0, inplace=True)
df

,line_str,title
line_num,,
0,The Project Gutenberg EBook of A Christmas Car...,"A Christmas Carol, by Charles Dickens"
1,,"A Christmas Carol, by Charles Dickens"
2,This eBook is for the use of anyone anywhere a...,"A Christmas Carol, by Charles Dickens"
3,almost no restrictions whatsoever. You may co...,"A Christmas Carol, by Charles Dickens"
4,re-use it under the terms of the Project Guten...,"A Christmas Carol, by Charles Dickens"
...,...,...
3971,,"A Christmas Carol, by Charles Dickens"
3972,This Web site includes information about Proje...,"A Christmas Carol, by Charles Dickens"
3973,including how to make donations to the Project...,"A Christmas Carol, by Charles Dickens"


In [15]:
a = df.line_str.str.match(r"\*\*\*\s*START OF (THE|THIS) PROJECT")
b = df.line_str.str.match(r"\*\*\*\s*END OF (THE|THIS) PROJECT")
an = df.loc[a].index[0]
bn = df.loc[b].index[0]
df = df.loc[an + 1 : bn - 2]

#---
# The other wise man
a2 = df2.line_str.str.match(r"\*\*\*\s*START OF (THE|THIS) PROJECT")
b2 = df2.line_str.str.match(r"\*\*\*\s*END OF (THE|THIS) PROJECT")
an2 = df2.loc[a2].index[0]
bn2 = df2.loc[b2].index[0]
df2 = df2.loc[an2 + 39 : bn2 - 2]

In [16]:
df2

,line_str,title
line_num,,
64,,"The Story of the Other Wise Man, by Henry van"
65,,"The Story of the Other Wise Man, by Henry van"
66,,"The Story of the Other Wise Man, by Henry van"
67,ILLUSTRATIONS,"The Story of the Other Wise Man, by Henry van"
68,,"The Story of the Other Wise Man, by Henry van"
...,...,...
1007,,"The Story of the Other Wise Man, by Henry van"
1008,His journey was ended. His treasures were acce...,"The Story of the Other Wise Man, by Henry van"
1009,had found the King.,"The Story of the Other Wise Man, by Henry van"


What is a Stave? 


"This is an unusual structure that mimics the way a musical piece is put together. The Staves follow the action of the story with the first stave setting the scene, the middle stave showing the turning point for Scrooge and the final stave concluding the story by presenting him as a changed man." -https://www.bbc.co.uk/bitesize/guides/zqxp4j6/revision/2


In [17]:
chap_lines = df.line_str.str.match(r"^\s*(STAVE )(ONE|TWO|THREE|FOUR|FIVE)(?!—)", case=False)
df.loc[chap_lines]

#Remove everything before STAVE ONE
chap_nums = [i+1 for i in range(df.loc[chap_lines].shape[0])]
df.loc[chap_lines, 'chap_num'] = chap_nums
df.chap_num = df.chap_num.ffill()
#Remove all lines containing [Illustration]


C:\Users\laure\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1684: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[key] = infer_fill_value(value)
C:\Users\laure\AppData\Local\Packages\PythonSoftwareFoundation.Python.3.8_qbz5n2kfra8p0\LocalCache\local-packages\Python38\site-packages\pandas\core\indexing.py:1773: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self._setitem_single_column(ilocs[0], value, p

In [18]:
chap_lines2 = df2.line_str.str.match(r"INTRODUCTION|THE SIGN IN THE SKY|BY THE WATERS OF BABYLON|FOR THE SAKE OF A LITTLE CHILD|IN THE HIDDEN WAY OF SORROW|A PEARL OF GREAT PRICE", case = True,)
df2.loc[chap_lines2]
chap_nums2 = [i+1 for i in range(df2.loc[chap_lines2].shape[0])]
df2.loc[chap_lines2, 'chap_num'] = chap_nums2
df2.chap_num= df2.chap_num.ffill()


In [19]:
df = df.loc[~df.chap_num.isna()] # Remove chapter heading lines
df = df.loc[~chap_lines] # Remove everything before Chapter 1
df.chap_num = df.chap_num.astype('int') # Convert chap_num from float to int

illustration_phrase =  df.line_str.str.match(r'\[Illustration:?.*| ?.*\]')
 #Remove book_results lines containing [Illustration]
df = df.loc[~illustration_phrase]


df2 = df2.loc[~df2.chap_num.isna()] # Remove chapter heading lines
df2 = df2.loc[~chap_lines2] # Remove everything before Chapter 1
df2.chap_num = df2.chap_num.astype('int') # Convert chap_num from float to int

illustration_phrase2 = df2.line_str.str.match(r'\[Illustration:?.*| ?.*\]')
 #Remove book_results lines c    ontaining [Illustration]
df2 = df2.loc[~illustration_phrase2]

In [22]:
df =pd.concat([df,df2])

In [24]:
dfc = df.groupby(OHCO[:2]).line_str.apply(lambda x: '\n'.join(x)).to_frame() # Make big string

In [25]:
dfp = dfc['line_str'].str.split(r'\n\n+', expand=True).stack()\
    .to_frame().rename(columns={0:'para_str'})
dfp.index.names = OHCO[:3]

dfp['para_str'] = dfp['para_str'].str.replace(r'\n', ' ').str.strip()
dfp = dfp[~dfp['para_str'].str.match(r'^\s*$')] # Remove empty paragraphs

C:\Users\laure\AppData\Local\Temp/ipykernel_29208/4088490130.py:5: FutureWarning: The default value of regex will change from True to False in a future version.
  dfp['para_str'] = dfp['para_str'].str.replace(r'\n', ' ').str.strip()


In [26]:
dfs = dfp['para_str'].str.split(r'[.?!;:"]+', expand=True).stack()\
    .to_frame().rename(columns={0:'sent_str'})

dfs.index.names = OHCO[:4]

dfs = dfs[~dfs['sent_str'].str.match(r'^\s*$')] # Remove empty paragraphs

In [27]:
dft = dfs['sent_str'].str.split(r"[\s',-]+", expand=True).stack()\
    .to_frame().rename(columns={0:'token_str'})
    
dft.index.names = OHCO[:5]
dft

token_str
title                                          chap_num para_num sent_num token_num           
 The Story of the Other Wise Man, by Henry van 1        0        0        0                You
                                                                          1               know
                                                                          2                the
                                                                          3              story
                                                                          4                 of
...                                                                                        ...
A Christmas Carol, by Charles Dickens          5        74       0        8          Christmas
                                                                          9              Carol
                                                                          10                by
                                                                          11           Charles
                                                                          12           Dickens

[41094 rows x 1 columns]

## Part 2: Annotate

In [29]:
def tokenize(doc_df, OHCO=OHCO, remove_pos_tuple=False, ws=False):
    
    # Paragraphs to Sentences
    df = doc_df.para_str\
        .apply(lambda x: pd.Series(nltk.sent_tokenize(x)))\
        .stack()\
        .to_frame()\
        .rename(columns={0:'sent_str'})
    
    # Sentences to Tokens
    # Local function to pick tokenizer
    def word_tokenize(x):
        if ws:
            s = pd.Series(nltk.pos_tag(nltk.WhitespaceTokenizer().tokenize(x)))
        else:
            s = pd.Series(nltk.pos_tag(nltk.word_tokenize(x))) # Discards stuff in between
        return s
            
    df = df.sent_str\
        .apply(word_tokenize)\
        .stack()\
        .to_frame()\
        .rename(columns={0:'pos_tuple'})
    
    # Grab info from tuple
    df['pos'] = df.pos_tuple.apply(lambda x: x[1])
    df['token_str'] = df.pos_tuple.apply(lambda x: x[0])
    if remove_pos_tuple:
        df = df.drop('pos_tuple', 1)
    
    # Add index
    df.index.names = OHCO
    
    return df

In [30]:
TOKEN = tokenize(dfp, ws=False)

In [31]:
TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')


C:\Users\laure\AppData\Local\Temp/ipykernel_29208/2045287459.py:1: FutureWarning: The default value of regex will change from True to False in a future version.
  TOKEN['term_str'] = TOKEN['token_str'].str.lower().str.replace('[\W_]', '')


In [32]:
VOCAB = TOKEN.term_str.value_counts().to_frame()\
    .rename(columns={'index':'term_str', 'term_str':'n'})\
    .sort_index().reset_index().rename(columns={'index':'term_str'})
VOCAB.index.name = 'term_id'
VOCAB['num'] = VOCAB.term_str.str.match("\d+").astype('int')
sw = pd.DataFrame(nltk.corpus.stopwords.words('english'), columns=['term_str'])
sw = sw.reset_index().set_index('term_str')
sw.columns = ['dummy']
sw.dummy = 1

In [33]:
VOCAB['stop'] = VOCAB.term_str.map(sw.dummy)
VOCAB['stop'] = VOCAB['stop'].fillna(0).astype('int')
VOCAB[VOCAB.stop == 1].sample(10)


,term_str,n,num,stop
term_id,,,,
5139,what,107,0,1
95,all,157,0,1
3132,only,40,0,1
1387,each,16,0,1
120,an,94,0,1
3177,over,38,0,1
4657,there,164,0,1
1320,down,72,0,1
592,by,148,0,1


In [34]:
from nltk.stem.porter import PorterStemmer
stemmer1 = PorterStemmer()
VOCAB['stem_porter'] = VOCAB.term_str.apply(stemmer1.stem)

from nltk.stem.snowball import SnowballStemmer
stemmer2 = SnowballStemmer("english")
VOCAB['stem_snowball'] = VOCAB.term_str.apply(stemmer2.stem)

from nltk.stem.lancaster import LancasterStemmer
stemmer3 = LancasterStemmer()
VOCAB['stem_lancaster'] = VOCAB.term_str.apply(stemmer3.stem)

In [35]:
def get_POS_Count(row):
    term = row['term_str']

    return TOKEN[TOKEN['term_str']== term].value_counts('pos').idxmax() 


In [36]:
VOCAB['pos_max'] = VOCAB.apply(get_POS_Count, axis=1)

In [57]:
if 'term_rank' not in VOCAB.columns:
    VOCAB = VOCAB.sort_values('n', ascending=False).reset_index()
    VOCAB.index.name = 'term_rank'
    VOCAB = VOCAB.reset_index()
    VOCAB = VOCAB.set_index('term_id')
    VOCAB['term_rank'] = VOCAB['term_rank'] + 1

## Part 3: Create

In [38]:
count_method = 'n' # 'c' or 'n' # n = n tokens, c = distinct token (term) count
tf_method = 'sum' # sum, max, log, double_norm, raw, binary
tf_norm_k = .5 # only used for double_norm
idf_method = 'standard' # standard, max, smooth
gradient_cmap = 'YlGnBu' # YlGn, GnBu, YlGnBu; For tables; see https://matplotlib.org/3.1.0/tutorials/colors/colormaps.html 

In [39]:
SENTS = OHCO[:4]
PARAS = OHCO[:3]
CHAPS = OHCO[:2]
BOOKS = OHCO[:1]

In [40]:
TOKEN['term_id'] = TOKEN.term_str.map(VOCAB.reset_index().set_index('term_str').term_id)

In [41]:
VOCAB['pos_max'] = TOKEN.groupby(['term_id', 'pos']).pos.count().unstack().idxmax(1)

In [42]:
VOCAB.sample(5)


,term_str,n,num,stop,stem_porter,stem_snowball,stem_lancaster,pos_max
term_id,,,,,,,,
2312,hustled,1,0,0,hustl,hustl,hustl,VBD
1496,establishments,1,0,0,establish,establish,est,NNS
2529,kinder,2,0,0,kinder,kinder,kind,JJR
2388,inexorable,1,0,0,inexor,inexor,inex,JJ
2194,helpless,3,0,0,helpless,helpless,helpless,JJ


In [46]:
def tfidif_matrix(df, bag, count, tf_method, idf_method):
    '''
    Write a function that returns a TFIDF matrix, with the following arguments:
    df - The tokens data frame to use.
    bag - The OHCO level to use, e.g. which "bag" to use.
    count - The type of count to use (e.g. binary counts are regular counts).
    tf_method - The type of TF to use.
    idf_method - The type of IDF to use.
    '''

    #Bag Of Words 
    BOW = TOKEN.groupby(bag+['term_id']).term_id.count()\
    .to_frame().rename(columns={'term_id':'n'})
    BOW['c'] = BOW.n.astype('bool').astype('int')

    #create count matrix 
    DTCM = BOW[count_method].unstack().fillna(0).astype('int')

    #Compute TF

    if tf_method == 'sum':
        TF = DTCM.T / DTCM.T.sum()
    elif tf_method == 'max':
        TF = DTCM.T / DTCM.T.max()
    elif tf_method == 'log':
        TF = np.log10(1 + DTCM.T)
    elif tf_method == 'raw':
        TF = DTCM.T
    elif tf_method == 'double_norm':
        TF = DTCM.T / DTCM.T.max()
        TF = tf_norm_k + (1 - tf_norm_k) * TF[TF > 0]
    elif tf_method == 'binary':
        TF = DTCM.T.astype('bool').astype('int')
    TF = TF.T

    #Compute DF
    DF = DTCM[DTCM > 0].count()
    #Compute IDF
    N = DTCM.shape[0]

    if idf_method == 'standard':
        IDF = np.log10(N / DF)
    elif idf_method == 'max':
        IDF = np.log10(DF.max() / DF) 
    elif idf_method == 'smooth':
        IDF = np.log10((1 + N) / (1 + DF)) + 1

    #Compute TDIDF
    TFIDF = TF * IDF

    return(DF, IDF, TF, TFIDF, BOW)

In [63]:
book_results = tfidif_matrix(TOKEN, BOOKS, count_method, tf_method, idf_method)
#Move things to their places
#OCAB['df'] = DF
#VOCAB['idf'] = IDF
VOCAB['df'] = book_results[0]
VOCAB['idf'] = book_results[1]

#BOW['tf'] = TF.stack()
#BOW['tfidf'] = TFIDF.stack()
BOW['tf'] = book_results[2].stack()
BOW['tfidf'] = book_results[3].stack()

#VOCAB['tfidf_sum'] = TFIDF.sum()
VOCAB['tfidf_sum'] = book_results[3].sum()
TOKEN['tf'] = book_results[2].stack()
TOKEN['tfidf'] = book_results[3].stack()

VOCAB['idf'] = book_results[1]
TFIDF = book_results[3]
VOCAB['tfidf_sum'] = book_results[3].sum()
VOCAB = VOCAB.query('pos_max != "NNP"')
VOCAB = VOCAB.sort_values('tfidf_sum', ascending=False).head(4000)
terms = VOCAB.index.tolist()
TFIDF = TFIDF[terms]

## Part 4:

In [67]:
#L2 Normalization
TFIDF = TFIDF.apply(lambda x: x / np.sqrt(np.square(x).sum()), 1)

#Cetner word vectors
TFIDF = TFIDF - TFIDF.mean()


#Compute Covariance Matrix
COV = TFIDF.T.dot(TFIDF) / (TFIDF.shape[0] - 1)

COV.iloc[:5,:10].style.background_gradient()

term_id,3071,4651,2493,4500,361,4385,196,3853,3106,3027
term_id,,,,,,,,,,
3071,0.127495,-0.100480,-0.054807,0.045617,0.040939,-0.045673,-0.045673,0.039769,0.039769,0.039769
4651,-0.100480,0.079188,0.043194,-0.035951,-0.032264,0.035995,0.035995,-0.031342,-0.031342,-0.031342
2493,-0.054807,0.043194,0.023560,-0.019610,-0.017599,0.019634,0.019634,-0.017096,-0.017096,-0.017096
4500,0.045617,-0.035951,-0.019610,0.016322,0.014648,-0.016342,-0.016342,0.014229,0.014229,0.014229
361,0.040939,-0.032264,-0.017599,0.014648,0.013145,-0.014665,-0.014665,0.012770,0.012770,0.012770


In [69]:
# Decompose the Matrix
%time eig_vals, eig_vecs = eigh(COV)

#Convert Eigen Data to Dataframes
TERM_IDX = COV.index
EIG_VEC = pd.DataFrame(eig_vecs, index=TERM_IDX, columns=TERM_IDX)
EIG_VAL = pd.DataFrame(eig_vals, index=TERM_IDX, columns=['eig_val'])
EIG_VAL.index.name = 'term_id'
EIG_VEC.iloc[:5, :10].style.background_gradient()

Wall time: 3.4 s


term_id,3071,4651,2493,4500,361,4385,196,3853,3106,3027
term_id,,,,,,,,,,
3071,-0.017512,-0.106180,0.094671,-0.086036,0.168280,0.085638,-0.139033,0.084879,0.148086,0.034471
4651,-0.339640,0.227700,0.150295,-0.476528,-0.146853,-0.043502,0.048183,-0.060975,-0.053396,-0.014473
2493,-0.091967,-0.551279,0.075256,0.224160,-0.039750,-0.052918,0.080400,-0.033875,-0.098912,-0.011342
4500,-0.106265,-0.282795,0.143901,-0.107754,-0.360146,-0.029946,0.085438,-0.256245,-0.076365,0.086509
361,-0.691428,0.074411,-0.216216,0.286725,-0.002345,0.034620,-0.075696,0.037322,0.091155,0.037572


## Part 5: Present